In [2]:
import netCDF4 as nc
import numpy as np
import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Constantes (las mismas que hemos usado) ---
target_test_file_full_path = "/home/f-caballero/UM/TIF3/convLSTM-project/convLSTM-Model/netCDF_samples/netCDF_Big_sample/netcdf_500x500x18/20100102/201001021/221837.nc"
CLIP_PHYSICAL_MIN_DBZ = -29.0
CLIP_PHYSICAL_MAX_DBZ = 65.0
OUTPUT_NC_SCALE_FACTOR = np.float32(0.5)
OUTPUT_NC_ADD_OFFSET = np.float32(33.5)
OUTPUT_NC_FILL_BYTE = np.int8(-128)
PHYSICAL_FILL_VALUE = (float(OUTPUT_NC_FILL_BYTE) * OUTPUT_NC_SCALE_FACTOR) + OUTPUT_NC_ADD_OFFSET

print("Celda 1: Configuración completada.")

Celda 1: Configuración completada.


In [3]:
try:
    with nc.Dataset(target_test_file_full_path, 'r') as ds_in:
        dbz_var_in = ds_in.variables['DBZ']
        data_raw = dbz_var_in[:]
        scale_in = getattr(dbz_var_in, 'scale_factor', 1.0)
        offset_in = getattr(dbz_var_in, 'add_offset', 0.0)
        fill_value_packed_in = getattr(dbz_var_in, '_FillValue', None)

        dbz_physical = data_raw.astype(np.float32) * scale_in + offset_in
        if fill_value_packed_in is not None:
            dbz_physical[data_raw == fill_value_packed_in] = np.nan
        
        dbz_3d = dbz_physical[0]

        print("--- ESTADO 1: Después de Cargar y Desempaquetar ---")
        print(f"Forma del array: {dbz_3d.shape}")
        print(f"Tipo de dato: {dbz_3d.dtype}")
        print(f"Número de NaNs: {np.isnan(dbz_3d).sum()}")
        # Usamos un try/except por si el array estuviera inesperadamente vacío de números
        try:
            print(f"Mínimo (ignorando NaNs): {np.nanmin(dbz_3d)}")
            print(f"Máximo (ignorando NaNs): {np.nanmax(dbz_3d)}")
        except ValueError:
            print("El array no contiene valores numéricos.")

except Exception as e:
    print(f"Error en la Celda 2: {e}")

--- ESTADO 1: Después de Cargar y Desempaquetar ---
Forma del array: (18, 500, 500)
Tipo de dato: float32
Número de NaNs: 0
Mínimo (ignorando NaNs): 19.0
Máximo (ignorando NaNs): 61.75


In [4]:
# Hacemos una copia para no modificar el original de la celda anterior
dbz_cleaned = dbz_3d.copy()

# Limpiamos ceros
are_close_to_zero = np.isclose(dbz_cleaned, 0.0)
num_zeros = np.count_nonzero(are_close_to_zero)
dbz_cleaned[are_close_to_zero] = np.nan

# Limpiamos valores bajos
below_min = dbz_cleaned < CLIP_PHYSICAL_MIN_DBZ
num_below_min = np.count_nonzero(below_min)
dbz_cleaned[below_min] = np.nan

print("--- ESTADO 2: Después de Limpiar 0s y valores bajos ---")
print(f"Se encontraron y eliminaron {num_zeros} ceros.")
print(f"Se encontraron y eliminaron {num_below_min} valores por debajo de {CLIP_PHYSICAL_MIN_DBZ} dBZ.")
print(f"Número de NaNs ahora: {np.isnan(dbz_cleaned).sum()}")
try:
    print(f"Mínimo (ignorando NaNs): {np.nanmin(dbz_cleaned)}")
    print(f"Máximo (ignorando NaNs): {np.nanmax(dbz_cleaned)}")
except ValueError:
    print("El array no contiene valores numéricos.")

--- ESTADO 2: Después de Limpiar 0s y valores bajos ---
Se encontraron y eliminaron 0 ceros.
Se encontraron y eliminaron 4454368 valores por debajo de -29.0 dBZ.
Número de NaNs ahora: 0
Mínimo (ignorando NaNs): 19.0
Máximo (ignorando NaNs): 61.75


In [5]:
dbz_clipped = np.clip(dbz_cleaned, CLIP_PHYSICAL_MIN_DBZ, CLIP_PHYSICAL_MAX_DBZ)

print("--- ESTADO 3: Después de np.clip ---")
print(f"Número de NaNs: {np.isnan(dbz_clipped).sum()}")
try:
    print(f"Mínimo (ignorando NaNs): {np.nanmin(dbz_clipped)}")
    print(f"Máximo (ignorando NaNs): {np.nanmax(dbz_clipped)}")
except ValueError:
    print("El array no contiene valores numéricos.")

--- ESTADO 3: Después de np.clip ---
Número de NaNs: 0
Mínimo (ignorando NaNs): 19.0
Máximo (ignorando NaNs): 61.75


In [6]:
dbz_for_packing = np.where(np.isnan(dbz_clipped), PHYSICAL_FILL_VALUE, dbz_clipped)

print("--- ESTADO 4: Después de np.where (listo para empaquetar) ---")
print("Este array ya NO debería tener NaNs.")
print(f"Número de NaNs: {np.isnan(dbz_for_packing).sum()}")
print(f"Mínimo (valor real): {np.min(dbz_for_packing)}")
print(f"Máximo (valor real): {np.max(dbz_for_packing)}")

--- ESTADO 4: Después de np.where (listo para empaquetar) ---
Este array ya NO debería tener NaNs.
Número de NaNs: 0
Mínimo (valor real): -30.5
Máximo (valor real): 61.75


In [7]:
dbz_packed_byte = np.round((dbz_for_packing - OUTPUT_NC_ADD_OFFSET) / OUTPUT_NC_SCALE_FACTOR).astype(np.int8)
# No aplicamos el clip de -127 a 127 aquí para ver los valores raw del cálculo

print("--- ESTADO 5: Después del cálculo de empaquetado ---")
print("Estos son los valores de byte que se escribirían en el archivo.")
print(f"Tipo de dato: {dbz_packed_byte.dtype}")
print(f"Mínimo (valor de byte): {np.min(dbz_packed_byte)}")
print(f"Máximo (valor de byte): {np.max(dbz_packed_byte)}")
print(f"Valor más frecuente: {np.bincount(dbz_packed_byte - np.min(dbz_packed_byte)).argmax() + np.min(dbz_packed_byte)}")

# Imprimimos una pequeña muestra del centro del array empaquetado
print("Muestra de datos empaquetados (centro del array):")
print(dbz_packed_byte[9, 250, 240:260])

--- ESTADO 5: Después del cálculo de empaquetado ---
Estos son los valores de byte que se escribirían en el archivo.
Tipo de dato: int8
Mínimo (valor de byte): -128
Máximo (valor de byte): 56


ValueError: object too deep for desired array